### https://techblog-history-younghunjo1.tistory.com/376
- 신경망 모델은 덧셈, 곱셈 연산도 존재하지만 가장 핵심은 Activation Function Operation 이다.
- 활성화 함수 종류에 따라 오차역전파 방법이 차이가 있다.

# 1. ReLU
- ReLU는 x가 0보다 작거나 같을 때, y를 모두 0으로 차단해버리는 활성화함수 이다.
- ReLU 함수의 x에 대한 y의 미분은 순전파 때의 입력 데이터가 0보다 컸다면, 이를 역전파 수행할 때 이전 노드에서 흘러들어온 국소적인 미분 값을 그대로 흘려보내는 것을 의미한다.
- 반대로 순전파 시 x가 0이었다면 이를 역전파 수행 시, 이전 노드에서 흘러들어온 국소적인 미분 값을 전달하지 않는 것을 의미한다. x가 0 이하일 때 미분값이 0이기 때문에 이전 노드에서 어떤 값이 흘러나왔던 간에 0을 곱하면 0이 되기 때문이다.

In [3]:
import numpy as np

# Relu 활성함수의 역전파 계층 만들기
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x: np.array):
        # 0보다 작거나 같은 값은 출력값 0처리,
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, d_out):
        d_out[self.mask] = 0
        dx = d_out
        
        return dx
    
datas = np.append(np.zeros(10), np.ones(10) + 1)
np.random.shuffle(datas)
print("Input - {}".format(datas))

relu_layer = Relu()
forward_result = relu_layer.forward(datas)
print("Forward Propagation - {}".format(forward_result))

back_result = relu_layer.backward(datas)
print("Backward Propagation - {}".format(back_result))

Input - [2. 0. 2. 0. 2. 2. 0. 0. 0. 0. 2. 0. 2. 0. 0. 2. 2. 2. 2. 0.]
Forward Propagation - [2. 0. 2. 0. 2. 2. 0. 0. 0. 0. 2. 0. 2. 0. 0. 2. 2. 2. 2. 0.]
Backward Propagation - [2. 0. 2. 0. 2. 2. 0. 0. 0. 0. 2. 0. 2. 0. 0. 2. 2. 2. 2. 0.]


# 2. Sigmoid
- 시그모이드 함수 연산은 계산 그래프(Computation Graph)로 펼쳐서 확인할 수 있다.
1. 나누기 연산
2. 덧셈 연산
3. exp(Exponential) 연산

In [4]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x: np.array):
        # sigmoid
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1.0 - self.out)
        return dx
    
datas = np.append(np.zeros(10), np.ones(10) + 1)
np.random.shuffle(datas)
print("Input - {}".format(datas))

sig_layer = Sigmoid()
forward_result = sig_layer.forward(datas)
print("Forward Propagation - {}".format(forward_result))

back_result = sig_layer.backward(datas)
print("Backward Propagation - {}".format(back_result))

Input - [2. 0. 2. 0. 0. 0. 2. 2. 0. 2. 0. 2. 2. 2. 2. 0. 2. 0. 0. 0.]
Forward Propagation - [0.88079708 0.5        0.88079708 0.5        0.5        0.5
 0.88079708 0.88079708 0.5        0.88079708 0.5        0.88079708
 0.88079708 0.88079708 0.88079708 0.5        0.88079708 0.5
 0.5        0.5       ]
Backward Propagation - [0.20998717 0.         0.20998717 0.         0.         0.
 0.20998717 0.20998717 0.         0.20998717 0.         0.20998717
 0.20998717 0.20998717 0.20998717 0.         0.20998717 0.
 0.         0.        ]


# 3. Affine
- Affine은 행렬의 곱을 의미한다.
    - 신경망은 Forward Propagation을 수행할 때, 행렬의 곱을 수행한다. X * W + b
    - Affine은 기하학에서 행렬의 곱을 의마한다고 해서, 행렬의 곱 연산에 대한 Forward Propagation 및 Back Propagation을 수행하는 계층을 Affine 계층이라고 부른다.
- Back propagation은 파라미터들의 변화량을 구함으로써, 기존 파라미터 값에 더해주거나 빼주기 위함이 목적이다. 그렇기 때문에 Back propagation의 결과로 얻은 각 파라미터의 변화량이 담긴 행렬의 shape가 파라미터의 shape와 동일해야 한다.
    - (3,) X (?, ?) = (2,)
    - -> (3,) X (3, 2) = (2,)
    - (?, ?) X (1, 3) = (2, 3)
    - -> (2, 1) X (1, 3) = (2, 3)
- 편향(Bias)의 고려 : 편향인 b의 미분값을 N개의 데이터마다 더해서 구하게 된다.

In [42]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.matmul(x, self.W) + self.b
        return out
    
    def backward(self, d_out):
        dx = np.dot(d_out, self.W.T)
        self.dW = np.dot(self.x.T, d_out)
        self.db = np.sum(d_out, axis=0)
        
        return dx

In [44]:
X = np.random.normal(size=(2,))
W = np.random.normal(size=(2,3))
b = np.random.normal(size=(3,))

print("Input", X)
print("Weights", W)
print("Bias", b)
print("------------------------")

affine_layer = Affine(W, b)
out = affine_layer.forward(X)
print("Output",out)
print("------------------------")

# dx = affine_layer.backward(out)
# print(dx)

Input [0.41753093 0.77283542]
Weights [[ 1.46118191 -0.92181319  0.59844233]
 [ 0.15405787 -0.92417    -0.17063844]]
Bias [-1.55900431 -1.43636483  0.71649845]
------------------------
Output [-0.82985429 -2.53548166  0.8344912 ]
------------------------


# 4. Softmax-with-Loss
- 여기서의 Loss는 Cross Entropy를 의미
- Softmax + Cross Entropy 함수가 함께 있는 계층
- Softmax 계층은 보통 신경망 모델을 학습시킬 때만 사용하고, 테스트 데이터에 대한 예측! 즉, '추론'하는 단계에서는 보통 사용하지 않는다.
    - softmax 계층은 예측 출력값과 실제 정답 간의 차이(손실 함수)를 0으로 만드는 파라미터 변화량을 찾는 즉, 학습할 때에 기여하기 때문이다.
    - Softmax 함수는 파라미터 최적화에 크게 기여
    - '추론' 단계에서는 단순히 출력값이 큰 것을 최종 출력으로 내뱉기만 하면 되므로, 손실함수를 건드릴 필요가 없다.
- Softmax with Loss는 계산 그래프의 구조는 복잡하지만, 최종적인 미분값이 y라는 예측값과 t라는 정답만을 갖고 계산할 수 있다라는 특징을 가진다. 손실함수로 Cross Entropy를 사용하기 때문이라고 한다. CEE가 그렇게 설계되어있다.

In [45]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # Prediction
        self.t = None # label
        
    def softmax(x):
        max_x = np.max(x)
        exp_x = np.exp(x - max_x)
        exp_X_sum = np.sum(exp_x)
        
        return exp_x / exp_x_sum
    
    def cross_entropy_error(y, t):
        if y.ndim == 1:
            t = t.reshape(1, t.size)
            y = y.reshape(1, y.size)
        
        # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
        if t.size == y.size:
            t = t.argmax(axis=1)
            
        batch_size = y.shape[0]
        return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
    
    def forward(self, x, t):
        self.t = t
        self.y = self.softmax(x)
        self.loss = self.cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx